### Imports and Setup

In [ ]:
import google.generativeai as genai
import os

import yaml

try:
    # Load the config file
    with open('config.yaml', 'rt') as f:
        config = yaml.safe_load(f.read())
except:
    print("Got an Error  While Reading Config")

genai.configure(api_key=config['gemini-api-key'])

## Utils and Prompts

In [ ]:

instructions_for_llm = """
You are a knowledgeable shoe recommendation assistant. Help the customer find the perfect shoes based on their preferences.

Important Instructions:
1. Ask interactive questions to understand user requirements fully
2. When you have gathered enough information about user preferences, call the product recommendation function
3. Keep your responses conversational but focused on gathering relevant information
4. Once you have clear requirements, summarize them and use the perform_search_wrapper function
5. Not only summarize the requirements but also add you best recommendation of shoes for customers requirements, 
    for example if a person asks for outdoor activiteis shoes,
      you can add your best recommendation according to situation as "Winter Hiking Boots" or "Hingh Ankle Boots" etc

Information to gather:
- Event/Occasion
- Primary use
- Style preferences
- Color preferences
- Gender
- Any specific requirements

- Limit you response to 20 words only be specific

When you have sufficient information, call the perform_search_wrapper function with a detailed but concise search query.
Do not mention the function or its calling in your responses to the user.
"""

def perform_search_wrapper(text_query: str):
    """Call this function to look for the desired shoes.
    
    Args:
        text_query: Input text query containing requirements of user for shoes curated by LLM
    """
    print("\nSearching for shoes with criteria:", text_query)
    # return {"status": "success", "query": text_query}

# Function declaration for Gemini
function_declarations = [{
    "name": "perform_search_wrapper",
    "description": "Search for shoes based on user requirements. \
        - When creating parameter for function call don't add search for or find for in the querry  ",
    "parameters": {
        "type": "object",
        "properties": {
            "text_query": {
                "type": "string",
                "description": "Comprehensive search query based on user requirements"
            }
        },
        "required": ["text_query"]
    }
}]

def initialize_chat():
    """Initialize the chat with the model and set up initial context"""
    model = genai.GenerativeModel("gemini-pro",
                                generation_config={
                                    "temperature": 0.7,
                                    "top_p": 0.8,
                                    "top_k": 40
                                })
    
    chat = model.start_chat(history=[
        {
            "role": "user",
            "parts": [instructions_for_llm]
        },
        {
            "role": "model",
            "parts": ["Hello! I'm your shoe shopping assistant. What type of shoes are you looking for today?"]
        }
    ])
    return chat

def process_response(response):
    """Process the model's response and handle function calls"""
# try:
    function_call = response.candidates[0].content.parts[0].function_call
    args = function_call.args
    if function_call.name == "perform_search_wrapper":
        # Parse arguments and call the function
        # args = json.loads(function_call.args)
        perform_search_wrapper(args["text_query"])
        return True
# except Exception as e:
    # print(f"Error processing response: {str(e)}")
    return False


In [ ]:
chat = initialize_chat()
print("\nWelcome to your Shoe Shopping Assistant! 👟")
print("Tell me about what kind of shoes you're looking for, and I'll help you find the perfect pair!")


while True:
    # try:
    # Get user input
    user_input =  str(input("\nYou: "))
    if user_input.strip().lower() in ['quit', 'exit', 'bye']:
        print("Thank you for shopping with us! Goodbye! 👋")
        break

    response = chat.send_message(
        user_input,
        tools=[{
            'function_declarations': function_declarations
        }]
    )
    print("*********************")
    print("user_input ", user_input)
    print("*********************")
    print(response)
    print("*********************")
    # Process any function calls
    has_function_call = process_response(response)
    print(has_function_call)
    if has_function_call:
        print("\nFound matching shoes based on your requirements! ✨")
        # Here you can add code to display actual product results
    else:
        print("*********************")
        print(response.text)
        print("*********************")

# except Exception as e:
    # print(f"\nAn error occurred: {str(e)}")
    print("Let's continue our conversation...")